## Chroma

In [ ]:
!pip install -q chromadb sentence-transformers PyPDF2 langchain langchain-community langchain-huggingface langchain-text-splitters accelerate ragas datasets 

# Setup and Retrieval

In [ ]:
# Step 1: Imports

import os
from pathlib import Path
from typing import List, Dict, Any
from langchain_community.vectorstores import Chroma
from sentence_transformers import SentenceTransformer, util
import re
import json
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from PyPDF2 import PdfReader
import time
!rm -rf course_db

In [ ]:
from huggingface_hub import login
#  Enter huggingface token here
login(token="*****")

# Select and one embedding model and comment out the rest
embedding_model = "sentence-transformers/all-MiniLM-L12-v2"
embedding_model = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
embedding_model = "sentence-transformers/distiluse-base-multilingual-cased-v2"
embedding_model = "intfloat/multilingual-e5-large"
# Embedding model
model = HuggingFaceEmbeddings(model_name=embedding_model)

persist_dir = "./course_db"
os.makedirs(persist_dir, exist_ok=True)

In [ ]:
# Step 2: Define PDF files and course mapping
# Update the path according to you current folder structure 
course_files = {
    "Master in AI-Driven Supply Chain Management": "Course_list/ai-driven-supply-chain-management-msc_html.pdf",
    "Master in Applied Psychology": "Course_list/applied-psychology-msc_html.pdf",
    "Master in Applied Research in Computer Science": "Course_list/applied-research-in-computer-science-msc_html.pdf",
    "Master in Artificial Intelligence Aided Mobility Design": "Course_list/artificial-intelligence-aided-mobility-design-ma_html.pdf",
    "Master in Artificial Intelligence and Robotics": "Course_list/artificial-intelligence-and-robotics-msc_html.pdf",
    "Bachlors in Business Administration": "Course_list/business-administration-ba_html.pdf",
    "Bachlors in Business Information Systems": "Course_list/business-information-systems-bsc_html.pdf",
    "Bachlors in Business Law": "Course_list/business-law-llb_html.pdf",
    "Bachlors in Communication Design": "Course_list/communication-design-ba_html.pdf",
    "Master in Compliance IT and Data Protection (LLM)": "Course_list/compliance-it-and-data-protection-llm_html.pdf",
    "Master in Compliance IT and Data Protection (MBA)": "Course_list/compliance-it-and-data-protection-mba_html.pdf",
    "Master in Composite Materials": "Course_list/composite-materials-meng_html.pdf",
    "Bachlors in Computer Science": "Course_list/computer-science-bsc_html.pdf",
    "Bachlors in Computer Science International": "Course_list/computer-science-international-bsc_html.pdf",
    "Master in Computer Science": "Course_list/computer-science-msc_html.pdf",
    "Master in Cross-Cultural Nursing Practice": "Course_list/cross-cultural-nursing-practice-msc_html.pdf",
    "Bachlors in Design and Mobility": "Course_list/design-and-mobility-ba_html.pdf",
    "Master in Digital Business Management": "Course_list/digital-business-management-msc_html.pdf",
    "Master in Digitalization and Innovation": "Course_list/digitalization-and-innovation-mba_html.pdf",
    "Bachlors in Economic and Organizational Sociology": "Course_list/economic-and-organizational-sociology-ba_html.pdf",
    "Bachlors in Economic Psychology": "Course_list/economic-psychology-bsc_html.pdf",
    "Bachlors in Engineering Science International": "Course_list/engineering-science-international-beng_html.pdf",
    "Bachlors in Engineering Sciences": "Course_list/engineering-sciences-beng.html.pdf",
    "Bachlors in Environmental Engineering": "Course_list/environmental-engineering-beng.html.pdf",
    "Master in General Management": "Course_list/general-management-mba_html.pdf",
    "Master in Global Management": "Course_list/global-management-ma_html.pdf",
    "Master in Human Resources and Organizational Management": "Course_list/human-resources-and-organizational-management_html.pdf",
    "Master in Information Systems in Public Administration": "Course_list/information-systems-in-public-administration-msc_html.pdf",
    "Bachlors in Innovative Healthcare": "Course_list/innovative-healthcare-bsc_html.pdf",
    "Bachlors in Innovative Textiles": "Course_list/innovative-textiles-beng_html.pdf",
    "Bachlors in International Management": "Course_list/international-management-ba_html.pdf",
    "Master in International Project Management": "Course_list/international-project-management-ma_html.pdf",
    "Master in Marketing Management": "Course_list/marketing-management-msc_html.pdf",
    "Master in Mechanical Engineering": "Course_list/mechanical-engineering-meng.html.pdf",
    "Bachlors in Media Informatics": "Course_list/media-informatics-bsc_html.pdf",
    "Bachlors in Mobile App Development": "Course_list/mobile-app-development-bsc_html.pdf",
    "Bachlors in Nursing (Nursing Profession Act)": "Course_list/nursing-bsc-according-to-the-nursing-profession-act_html.pdf",
    "Bachlors in Nursing (Experienced Professionals)": "Course_list/nursing-bsc-for-experienced-professionals_html.pdf",
    "Master in Operational Excellence": "Course_list/operational-excellence-mba-and-eng_html.pdf",
    "Master in Smart Society": "Course_list/smart-society-msc_html.pdf",
    "Master in Software Engineering for Industrial Applications": "Course_list/software-engineering-for-industrial-applications-meng.html.pdf",
    "Master in Supply Chain Management": "Course_list/supply-chain-management-msc_html.pdf",
    "Master in Sustainable Engineering and Project Management": "Course_list/sustainable-engineering-and-project-management-mba-and-eng_html.pdf",
    "Master in Sustainable Textiles": "Course_list/sustainable-textiles-meng_html.pdf",
    "Master in Sustainable Water Management and Engineering": "Course_list/sustainable-water-management-and-engineering-meng_html.pdf",
    "Bachlors in Textile Design": "Course_list/textile-design-ba_html.pdf",
    "Master in Textile Design": "Course_list/textile-design-ma_html.pdf",
    "Bachlors in Vocational Education in Health Care": "Course_list/vocational-education-in-health-care-ba.html.pdf"
}
course_names = list(course_files.keys())
course_embs = model.embed_documents(course_names)

In [ ]:
# Step 3: Define function for chunking

def read_pdf_chunks(filepath, chunk_size=700, overlap=200):
    """
    Reads text from a PDF and splits it into semantically meaningful chunks.
    This uses LangChain's RecursiveCharacterTextSplitter, which respects sentence and paragraph boundaries.
    """
    reader = PdfReader(filepath)
    full_text = ""

    # Combine text from all pages
    for page in reader.pages:
        page_text = page.extract_text() or ""
        full_text += page_text + "\n"

    # Use RecursiveCharacterTextSplitter for semantic chunking
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=overlap,
        length_function=len,
        separators=["\n\n", ".", "!", "?", " ", ""]
    )

    chunks = text_splitter.split_text(full_text)
    return chunks


In [ ]:
# Step 4: Build Chroma Database with metadata for all courses

vector_store = Chroma(
    collection_name="course_docs",
    persist_directory=persist_dir,
    embedding_function=model
)

def add_pdf_to_chromadb(course_name, pdf_path, total_indexing_time):
  """Add PDF text chunks and metadata to Chroma collection."""
  start_indexing_time = time.time()
  chunks = read_pdf_chunks(pdf_path)


  ids = [f"{course_name}_{i}" for i in range(len(chunks))]
  metadatas = [{"course": course_name, "filename": pdf_path}] * len(chunks)

  vector_store.add_texts(
      texts=chunks,
      metadatas=metadatas,
      ids=ids
  )

  end_indexing_time = time.time()
  total_indexing_time += (end_indexing_time - start_indexing_time)

  print(f"Added {len(chunks)} chunks for '{course_name}'")
  return total_indexing_time
# Add all PDFs to the ChromaDB collection and measure indexing time

total_indexing_time = 0
for name, path in course_files.items():
    total_indexing_time = add_pdf_to_chromadb(name, path, total_indexing_time)

vector_store.persist()


In [ ]:
# Step 5: Define function for detecting Course Name and cleaning query

def detect_course_and_clean_query(query, min_course_score=0.35, min_span_score=0.25, max_ngram=6):
  """Detect course name from query and remove its span."""
  query_emb = model.embed_documents([query])[0]
  cos_scores = util.cos_sim(query_emb, course_embs)[0]

  best_idx = int(cos_scores.argmax().item())
  best_course = course_names[best_idx]
  best_course_score = float(cos_scores[best_idx].item())

  if best_course_score < min_course_score:
      return None, query

  # Find matching n-gram for course name
  tokens = query.split()
  candidates = [" ".join(tokens[i:i+L]) for L in range(1, max_ngram+1) for i in range(len(tokens)-L+1)]
  cand_embs = model.embed_documents(candidates)
  cand_scores = util.cos_sim(course_embs[best_idx], cand_embs)[0]

  best_cand = candidates[int(cand_scores.argmax().item())]
  best_cand_score = float(cand_scores.max().item())

  cleaned_query = query
  if best_cand_score > min_span_score:
      pattern = re.escape(best_cand)
      cleaned_query = re.sub(pattern, "", query, flags=re.IGNORECASE, count=1).strip()
      cleaned_query = re.sub(r'\b(for|of|about|in|on)\b\s*([?.!,;:]?)$', r'\2', cleaned_query, flags=re.IGNORECASE).strip()

  return best_course, cleaned_query


In [ ]:
# Step 6: Retrieve from the detected course index

# Helper: build filtered retriever like FAISS version
def get_filtered_retriever(course_name=None, k=5):
    if course_name:
        return vector_store.as_retriever(
            search_type="similarity",
            search_kwargs={"k": k, "filter": {"course": course_name}}
        )
    else:
        return vector_store.as_retriever(
            search_type="similarity",
            search_kwargs={"k": k}
        )

def retrieve(query, k=3):
    # Load your persisted Chroma collection using LangChain wrapper
    vector_store = Chroma(
        collection_name="course_docs",
        persist_directory="./course_db",
        embedding_function=model
    )
    start_time = time.time()

    # Course detection logic
    course, clean_query = detect_course_and_clean_query(query)

    if course:
        print(f"🎯 Matched course (by similarity): {course}")
        retriever = get_filtered_retriever(course, k)
        docs = retriever.invoke(clean_query)
    else:
        print("⚠️ No strong course match — performing global search.")
        retriever = get_filtered_retriever(None, k)
        docs = retriever.invoke(clean_query)
        
    results = [(doc.metadata.get("source", 'unknown'), doc.page_content) for doc in docs]
    retrieval_time = time.time() - start_time


    print(f"⏱️ Retrieval time: {time.time() - start_time:.2f}s — {len(results)} chunks returned.")
    return results, retrieval_time


In [ ]:
# Step 7: Test the Retriver

query = "what are Language requirements for the course AI-Driven Supply Chain Management?"
results, retrieval_time  = retrieve(query, k=3)

for i, (course, text) in enumerate(results, 1):
    print(f"\n--- Result {i} (Course: {course}) ---\n{text[:800]}")

# Generation

In [ ]:
# Step 8: Create a prompt template

from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("""
You are an expert assistant. Answer the question using ONLY the provided context.
If the answer is not in the context, say you don't know.

QUESTION:
{question}

CONTEXT:
{context}

ANSWER:
""")


In [ ]:
# Step 9: Define LLM

from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline
from transformers import AutoTokenizer, pipeline, AutoModelForCausalLM

# 1. model_id = "meta-llama/Llama-3.1-8B-Instruct"
# 2. model_id = "mistralai/Ministral-8B-Instruct-2410"
model_id = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"

tokenizer = AutoTokenizer.from_pretrained(model_id)
llm_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map = "auto",
    dtype = "auto",
)

hf_pipeline = pipeline(
    "text-generation",
    model=llm_model,
    tokenizer=tokenizer,
    max_new_tokens=512,
)

# Create HuggingFacePipeline instance
hf_llm = HuggingFacePipeline(pipeline=hf_pipeline)

# Pass the HuggingFacePipeline instance as the 'llm' argument to ChatHuggingFace
llm = ChatHuggingFace(llm=hf_llm)


In [ ]:
# Step 10: Build a langchain
from langchain_core.runnables import RunnableParallel

rag_chain = (RunnableParallel({
    "context" : lambda x: retrieve(x["question"], k=3),
    "question" : lambda x: x["question"]
}))|prompt|llm

In [ ]:
# Step 11: Test the chain
response = rag_chain.invoke({"question": "Give me Program objectives for ai in supply chain?"})
print(response.content)

# MRR

In [ ]:

def is_relevant(chunk_text, ground_truth, threshold=0.6):
    """
    Returns True if retrieved chunk is semantically similar
    to the ground-truth answer.
    """
    emb_chunk = model.embed_documents([chunk_text])[0]
    emb_gt = model.embed_documents([ground_truth])[0]
    score = util.cos_sim(emb_chunk, emb_gt).item()
    return score >= threshold


In [ ]:
def mrr_semantic(retrieved_texts, ground_truth, threshold=0.6):
    for rank, text in enumerate(retrieved_texts, start=1):
        if is_relevant(text, ground_truth, threshold):
            return 1.0 / rank
    return 0.0

In [ ]:
from tqdm import tqdm

with open("raga_eval_dataset_english.json", "r", encoding="utf-8") as f:
    eval_data = json.load(f)

records = []

for sample in tqdm(eval_data):
    query = sample["query"]
    ground_truth = sample["ground_truth"]

    results, retrieval_time = retrieve(query, k=5)

    retrieved_texts = [text for _, text in results]

    mrr = mrr_semantic(retrieved_texts, ground_truth)

    records.append({
        "query": query,
        "MRR": mrr,
    })


In [ ]:
import pandas as pd
df = pd.DataFrame(records)

Mean_Recall_Rate = {
    "MRR": df["MRR"].mean()
}

print(Mean_Recall_Rate)

# RAGAS

In [ ]:
# Step 12: Import the RAGAS

from ragas import evaluate
import json
from ragas.metrics import (
    ContextPrecision,
    ContextRecall,
    Faithfulness,
    ResponseGroundedness,
    AnswerAccuracy,
    SemanticSimilarity
)

from datasets import Dataset


In [ ]:
# Step 13: Define function to execute the query without using langchain
def run_rag(question, k=3):
    results, retrieval_time = retrieve(question, k)
    contexts = [text for _, text in results]

    prompt_text = prompt.format(
        question=question,
        context="\n\n".join(contexts)
    )

    llm_start = time.time()
    response = llm.invoke(prompt_text)
    llm_time = time.time() - llm_start

    return {
        "answer": response.content,
        "contexts": contexts,
        "retrieval_time": retrieval_time,
        "llm_time": llm_time,
        "total_time": retrieval_time + llm_time
    }


In [ ]:
# Step 14a: Import the Evaluation data and evaluate it

with open("raga_eval_dataset_english.json", "r") as f:
    eval_data = json.load(f)

ragas_records = []
total_retrieval_time = 0
total_generation_time = 0

for item in eval_data:
    result = run_rag(item["query"], k=3)
    total_retrieval_time = total_retrieval_time + result["retrieval_time"]
    total_generation_time = total_generation_time + result["llm_time"]

    answer = result["answer"]
    contexts = result["contexts"]

    ragas_records.append({
        "question": item["query"],
        "answer": answer,
        "contexts": contexts,
        "ground_truth": item["ground_truth"]
    })

# Convert to HuggingFace Dataset
ragas_dataset = Dataset.from_list(ragas_records)


In [ ]:
# Step 14b: Import the Evaluation data and evaluate it

with open("raga_eval_dataset_english.json", "r") as f:
    eval_data = json.load(f)

ragas_records_sematic = []

for item in eval_data:
    result = run_rag(item["query"], k=3)
    
    answer = result["answer"]
    contexts = result["contexts"]

    ragas_records_sematic.append({
        "question": item["query"],
        "answer": answer,
        "contexts": contexts,
        "ground_truth": item["ground_truth"]
    })

# Convert to HuggingFace Dataset
ragas_records_sematic = Dataset.from_list(ragas_records_sematic)


In [ ]:
import os
# Enter Open AI Api key here
os.environ["OPENAI_API_KEY"] = "***"

In [ ]:
# Step 15a: Evaluate the RAG

from ragas.run_config import RunConfig 
results1 = evaluate(
    dataset=ragas_dataset,
    metrics=[
        ContextPrecision(),
        ContextRecall(),
        Faithfulness(),
    ],
    embeddings=model,
    run_config=RunConfig(max_workers=1)
)

print(results1)


In [ ]:
# Step 15b: Evaluate the RAG

results2 = evaluate(
    dataset=ragas_records_sematic,
    metrics=[
        SemanticSimilarity()
    ],
    embeddings=model,
    llm = llm,
    run_config=RunConfig(max_workers=1)
)

print(results2)

# Output

In [ ]:
print(f"For Embedding model:{embedding_model} and LLM {model_id}") 
print(f"Total Retrieval Time: {total_retrieval_time}")
print(f"Total Generation Time per query: {total_generation_time}")
print(f"Total Indexing Time: {total_indexing_time}")
print(f"Mean Recall Rate: {Mean_Recall_Rate.values()}")
print("RAGAS metrics")
print(results1)
print(results2)


In [ ]:
import os

def get_folder_size(path):
    total = 0
    for dirpath, _, filenames in os.walk(path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            total += os.path.getsize(fp)
    return total

size_bytes = get_folder_size("course_db")
print(f"Size: {size_bytes / (1024**2)} MB")